In [0]:
### This code analyzes fire incidents in San Francisco
### Steps
### Get CSV
### Create table and rename columns
### Repartition (Qty) and byPeriod?
### Convert to Parquet
### Load Parquet
### Compare Load time CSV x Parquet
### Exploratory Analysis
### Use Spark SQL - Group By + Sort Operations
### Test ML Library?
### Test Graphiz?
### Test Spark Streaming?

In [0]:
#Import libs
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType, TimestampType #Used for schema
from pyspark.sql.functions import * # in order to use Date Functions - Ex.: extract Year from Date
#from pyspark.sql.functions import to_timestamp

In [0]:
schema = StructType([
StructField("CallNumber",StringType(),True),
StructField("UnitID",StringType(),True),
StructField("IncidentNumber",StringType(),True),
StructField("CallType",StringType(),True),
StructField("CallDate",StringType(),True),
StructField("WatchDate",StringType(),True),
StructField("ReceivedDtTm",StringType(),True),
StructField("EntryDtTm",StringType(),True),
StructField("DispatchDtTm",StringType(),True),
StructField("ResponseDtTm",StringType(),True),
StructField("OnSceneDtTm",StringType(),True),
StructField("TransportDtTm",StringType(),True),
StructField("HospitalDtTm",StringType(),True),
StructField("CallFinalDisposition",StringType(),True),
StructField("AvailableDtTm",StringType(),True),
StructField("Address",StringType(),True),
StructField("City",StringType(),True),
StructField("ZipcodeofIncident",StringType(),True),
StructField("Battalion",StringType(),True),
StructField("StationArea",StringType(),True),
StructField("Box",StringType(),True),
StructField("OriginalPriority",StringType(),True),
StructField("Priority",StringType(),True),
StructField("FinalPriority",StringType(),True),
StructField("ALSUnit",StringType(),True),
StructField("CallTypeGroup",StringType(),True),
StructField("NumberofAlarms",StringType(),True),
StructField("UnitType",StringType(),True),
StructField("Unitsequenceincalldispatch",StringType(),True),
StructField("FirePreventionDistrict",StringType(),True),
StructField("SupervisorDistrict",StringType(),True),
StructField("Neighborhooods-AnalysisBoundaries",StringType(),True),
StructField("RowID",StringType(),True),
StructField("case_location",StringType(),True),
StructField("AnalysisNeighborhoods",StringType(),True)
  ])

In [0]:
#Get the file on this link
#https://data.sfgov.org/Public-Safety/Fire-Department-Calls-for-Service/nuek-vuh3
  
# File location and type
file_location = "/FileStore/tables/Fire_Department_Calls_for_Service.csv"
file_type = "csv"

# CSV options
infer_schema = "false"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
df = spark.read.format(file_type) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .schema(schema) \
  .load(file_location)

display(df)

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,ReceivedDtTm,EntryDtTm,DispatchDtTm,ResponseDtTm,OnSceneDtTm,TransportDtTm,HospitalDtTm,CallFinalDisposition,AvailableDtTm,Address,City,ZipcodeofIncident,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumberofAlarms,UnitType,Unitsequenceincalldispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhooods-AnalysisBoundaries,RowID,case_location,AnalysisNeighborhoods
210690030,T03,21030278,Alarms,03/10/2021,03/09/2021,03/10/2021 12:16:03 AM,03/10/2021 12:18:36 AM,03/10/2021 12:19:01 AM,03/10/2021 12:20:15 AM,null,null,null,Fire,03/10/2021 12:28:42 AM,1300 Block of MARKET ST,San Francisco,94102,B02,36,3111,3,3,3,false,Alarm,1,TRUCK,3,2,6,Tenderloin,210690030-T03,POINT (-122.41697932641094 37.7770834641944),36
210391607,E19,21017645,Alarms,02/08/2021,02/08/2021,02/08/2021 01:00:14 PM,02/08/2021 01:01:36 PM,02/08/2021 01:01:40 PM,02/08/2021 01:03:21 PM,02/08/2021 01:05:44 PM,null,null,Fire,02/08/2021 01:18:09 PM,400 Block of SERRANO DR,San Francisco,94132,B08,19,8581,3,3,3,true,Alarm,1,ENGINE,1,8,7,Lakeshore,210391607-E19,POINT (-122.48045074945836 37.7190118676788),16
210391164,T04,21017596,Alarms,02/08/2021,02/08/2021,02/08/2021 10:54:56 AM,02/08/2021 10:56:50 AM,02/08/2021 10:56:57 AM,02/08/2021 10:57:07 AM,02/08/2021 10:59:34 AM,null,null,Fire,02/08/2021 11:06:42 AM,600 Block of LONG BRIDGE ST,San Francisco,94158,B03,04,2264,3,3,3,false,Alarm,1,TRUCK,1,3,6,Mission Bay,210391164-T04,POINT (-122.39227179213904 37.77288298280324),4
210683008,T03,21030233,Structure Fire,03/09/2021,03/09/2021,03/09/2021 08:57:47 PM,03/09/2021 08:59:01 PM,03/09/2021 08:59:04 PM,03/09/2021 08:59:28 PM,03/09/2021 09:01:48 PM,null,null,Fire,03/09/2021 09:05:42 PM,600 Block of POLK ST,San Francisco,94102,B02,03,3114,3,3,3,false,Alarm,1,TRUCK,1,2,6,Tenderloin,210683008-T03,POINT (-122.4190191572421 37.78264403568861),36
210391034,E16,21017578,Citizen Assist / Service Call,02/08/2021,02/08/2021,02/08/2021 10:18:53 AM,02/08/2021 10:19:52 AM,02/08/2021 10:19:58 AM,02/08/2021 10:20:42 AM,02/08/2021 10:27:42 AM,null,null,Fire,02/08/2021 10:53:27 AM,FRANKLIN ST/FILBERT ST,San Francisco,94123,B04,16,3233,3,3,3,true,Alarm,1,ENGINE,1,4,2,Marina,210391034-E16,POINT (-122.42581353320875 37.79927566930728),13
210390767,T19,21017552,Other,02/08/2021,02/08/2021,02/08/2021 08:50:27 AM,02/08/2021 08:54:27 AM,02/08/2021 08:55:28 AM,02/08/2021 08:57:51 AM,null,null,null,Fire,02/08/2021 09:02:22 AM,"CALL BOX: JOHN DALY BL/MISSION ST,DC",Daly City,null,B09,33,9922,3,3,3,true,Fire,1,TRUCK,9,None,None,None,210390767-T19,POINT (-122.46239390119047 37.7049649190675),null
210681495,T03,21030060,Alarms,03/09/2021,03/09/2021,03/09/2021 12:47:08 PM,03/09/2021 12:49:37 PM,03/09/2021 12:49:48 PM,03/09/2021 12:51:10 PM,03/09/2021 12:53:01 PM,null,null,Fire,03/09/2021 12:58:21 PM,900 Block of SUTTER ST,San Francisco,94109,B04,03,1557,3,3,3,false,Alarm,1,TRUCK,1,4,3,Nob Hill,210681495-T03,POINT (-122.4160172278388 37.788337139908855),21
210681261,E08,21030034,Alarms,03/09/2021,03/09/2021,03/09/2021 11:37:16 AM,03/09/2021 11:39:15 AM,03/09/2021 11:39:29 AM,03/09/2021 11:40:38 AM,null,null,null,Fire,03/09/2021 11:41:28 AM,100 Block of BRANNAN ST,San Francisco,94107,B03,35,2134,3,3,3,true,Alarm,1,ENGINE,4,3,6,Financial District/South Beach,210681261-E08,POINT (-122.38904057366992 37.78432123494197),8
210382984,B05,21017398,Alarms,02/07/2021,02/07/2021,02/07/2021 09:18:38 PM,02/07/2021 09:20:02 PM,02/07/2021 09:21:15 PM,02/07/2021 09:21:15 PM,02/07/2021 09:25:11 PM,null,null,Fire,02/07/2021 09:35:06 PM,2100 Block of FELL ST,San Francisco,94117,B05,12,4554,3,3,3,false,Alarm,1,CHIEF,2,5,5,Lone Mountain/USF,210382984-B05,POINT (-122.45328305705388 37.77213783914884),18
210382403,T05,21017307,Alarms,02/07/2021,02/07/2021,02/07/2021 05:24:48 PM,02/07/2021 05:26:12 PM,02/07/2021 05:26:24 PM,02/07/2021 05:27:46 PM,02/07/2021 05:30:04 PM,null,null,Fire,02/07/2021 05:50:46 PM,1400 Block of GEARY BL

In [0]:
df.printSchema()

root
-- CallNumber: string (nullable = true)
-- UnitID: string (nullable = true)
-- IncidentNumber: string (nullable = true)
-- CallType: string (nullable = true)
-- CallDate: string (nullable = true)
-- WatchDate: timestamp (nullable = true)
-- ReceivedDtTm: timestamp (nullable = true)
-- EntryDtTm: timestamp (nullable = true)
-- DispatchDtTm: timestamp (nullable = true)
-- ResponseDtTm: timestamp (nullable = true)
-- OnSceneDtTm: timestamp (nullable = true)
-- TransportDtTm: timestamp (nullable = true)
-- HospitalDtTm: timestamp (nullable = true)
-- CallFinalDisposition: string (nullable = true)
-- AvailableDtTm: timestamp (nullable = true)
-- Address: string (nullable = true)
-- City: string (nullable = true)
-- ZipcodeofIncident: string (nullable = true)
-- Battalion: string (nullable = true)
-- StationArea: string (nullable = true)
-- Box: string (nullable = true)
-- OriginalPriority: string (nullable = true)
-- Priority: string (nullable = true)
-- FinalPriority: string (nullable = true)
-- ALSUnit: string (nullable = true)
-- CallTypeGroup: string (nullable = true)
-- NumberofAlarms: string (nullable = true)
-- UnitType: string (nullable = true)
-- Unitsequenceincalldispatch: string (nullable = true)
-- FirePreventionDistrict: string (nullable = true)
-- SupervisorDistrict: string (nullable = true)
-- Neighborhooods-AnalysisBoundaries: string (nullable = true)
-- RowID: string (nullable = true)
-- case_location: string (nullable = true)
-- AnalysisNeighborhoods: string (nullable = true)

In [0]:
from_pattern = "MM/dd/yyyy"
to_pattern = "yyyy-MM-dd"

In [0]:
df = df.withColumn("CallDateTs",to_timestamp(df["CallDate"], from_pattern))
df.display()

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,ReceivedDtTm,EntryDtTm,DispatchDtTm,ResponseDtTm,OnSceneDtTm,TransportDtTm,HospitalDtTm,CallFinalDisposition,AvailableDtTm,Address,City,ZipcodeofIncident,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumberofAlarms,UnitType,Unitsequenceincalldispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhooods-AnalysisBoundaries,RowID,case_location,AnalysisNeighborhoods,CallDateTs
210690030,T03,21030278,Alarms,03/10/2021,03/09/2021,03/10/2021 12:16:03 AM,03/10/2021 12:18:36 AM,03/10/2021 12:19:01 AM,03/10/2021 12:20:15 AM,null,null,null,Fire,03/10/2021 12:28:42 AM,1300 Block of MARKET ST,San Francisco,94102,B02,36,3111,3,3,3,false,Alarm,1,TRUCK,3,2,6,Tenderloin,210690030-T03,POINT (-122.41697932641094 37.7770834641944),36,2021-03-10T00:00:00.000+0000
210391607,E19,21017645,Alarms,02/08/2021,02/08/2021,02/08/2021 01:00:14 PM,02/08/2021 01:01:36 PM,02/08/2021 01:01:40 PM,02/08/2021 01:03:21 PM,02/08/2021 01:05:44 PM,null,null,Fire,02/08/2021 01:18:09 PM,400 Block of SERRANO DR,San Francisco,94132,B08,19,8581,3,3,3,true,Alarm,1,ENGINE,1,8,7,Lakeshore,210391607-E19,POINT (-122.48045074945836 37.7190118676788),16,2021-02-08T00:00:00.000+0000
210391164,T04,21017596,Alarms,02/08/2021,02/08/2021,02/08/2021 10:54:56 AM,02/08/2021 10:56:50 AM,02/08/2021 10:56:57 AM,02/08/2021 10:57:07 AM,02/08/2021 10:59:34 AM,null,null,Fire,02/08/2021 11:06:42 AM,600 Block of LONG BRIDGE ST,San Francisco,94158,B03,04,2264,3,3,3,false,Alarm,1,TRUCK,1,3,6,Mission Bay,210391164-T04,POINT (-122.39227179213904 37.77288298280324),4,2021-02-08T00:00:00.000+0000
210683008,T03,21030233,Structure Fire,03/09/2021,03/09/2021,03/09/2021 08:57:47 PM,03/09/2021 08:59:01 PM,03/09/2021 08:59:04 PM,03/09/2021 08:59:28 PM,03/09/2021 09:01:48 PM,null,null,Fire,03/09/2021 09:05:42 PM,600 Block of POLK ST,San Francisco,94102,B02,03,3114,3,3,3,false,Alarm,1,TRUCK,1,2,6,Tenderloin,210683008-T03,POINT (-122.4190191572421 37.78264403568861),36,2021-03-09T00:00:00.000+0000
210391034,E16,21017578,Citizen Assist / Service Call,02/08/2021,02/08/2021,02/08/2021 10:18:53 AM,02/08/2021 10:19:52 AM,02/08/2021 10:19:58 AM,02/08/2021 10:20:42 AM,02/08/2021 10:27:42 AM,null,null,Fire,02/08/2021 10:53:27 AM,FRANKLIN ST/FILBERT ST,San Francisco,94123,B04,16,3233,3,3,3,true,Alarm,1,ENGINE,1,4,2,Marina,210391034-E16,POINT (-122.42581353320875 37.79927566930728),13,2021-02-08T00:00:00.000+0000
210390767,T19,21017552,Other,02/08/2021,02/08/2021,02/08/2021 08:50:27 AM,02/08/2021 08:54:27 AM,02/08/2021 08:55:28 AM,02/08/2021 08:57:51 AM,null,null,null,Fire,02/08/2021 09:02:22 AM,"CALL BOX: JOHN DALY BL/MISSION ST,DC",Daly City,null,B09,33,9922,3,3,3,true,Fire,1,TRUCK,9,None,None,None,210390767-T19,POINT (-122.46239390119047 37.7049649190675),null,2021-02-08T00:00:00.000+0000
210681495,T03,21030060,Alarms,03/09/2021,03/09/2021,03/09/2021 12:47:08 PM,03/09/2021 12:49:37 PM,03/09/2021 12:49:48 PM,03/09/2021 12:51:10 PM,03/09/2021 12:53:01 PM,null,null,Fire,03/09/2021 12:58:21 PM,900 Block of SUTTER ST,San Francisco,94109,B04,03,1557,3,3,3,false,Alarm,1,TRUCK,1,4,3,Nob Hill,210681495-T03,POINT (-122.4160172278388 37.788337139908855),21,2021-03-09T00:00:00.000+0000
210681261,E08,21030034,Alarms,03/09/2021,03/09/2021,03/09/2021 11:37:16 AM,03/09/2021 11:39:15 AM,03/09/2021 11:39:29 AM,03/09/2021 11:40:38 AM,null,null,null,Fire,03/09/2021 11:41:28 AM,100 Block of BRANNAN ST,San Francisco,94107,B03,35,2134,3,3,3,true,Alarm,1,ENGINE,4,3,6,Financial District/South Beach,210681261-E08,POINT (-122.38904057366992 37.78432123494197),8,2021-03-09T00:00:00.000+0000
210382984,B05,21017398,Alarms,02/07/2021,02/07/2021,02/07/2021 09:18:38 PM,02/07/2021 09:20:02 PM,02/07/2021 09:21:15 PM,02/07/2021 09:21:15 PM,02/07/2021 09:25:11 PM,null,null,Fire,02/07/2021 09:35:06 PM,2100 Block of FELL ST,San Francisco,94117,B05,12,4554,3,3,3,false,Alarm,1,CHIEF,2,5,5,Lone Mountain/USF,210382984-B05,POINT (-122.45328305705388 37.772

In [0]:
df.groupby(year("CallDateTs")).count().show()

+----------------+------+
year(CallDateTs)| count|
+----------------+------+
 2003|240457|
 2007|235856|
 2018|310700|
 2015|296244|
 2006|235440|
 2013|273097|
 2014|280905|
 2019|319855|
 2004|235353|
 2020|285743|
 2012|265596|
 2009|244633|
 2016|303967|
 2001|220328|
 2005|232919|
 2000|158154|
 2010|256174|
 2011|268689|
 2008|249690|
 2017|312471|
+----------------+------+
only showing top 20 rows

In [0]:
df = df.withColumn("CallYear",year(df["CallDateTs"]))
df.display()

CallNumber,UnitID,IncidentNumber,CallType,CallDate,WatchDate,ReceivedDtTm,EntryDtTm,DispatchDtTm,ResponseDtTm,OnSceneDtTm,TransportDtTm,HospitalDtTm,CallFinalDisposition,AvailableDtTm,Address,City,ZipcodeofIncident,Battalion,StationArea,Box,OriginalPriority,Priority,FinalPriority,ALSUnit,CallTypeGroup,NumberofAlarms,UnitType,Unitsequenceincalldispatch,FirePreventionDistrict,SupervisorDistrict,Neighborhooods-AnalysisBoundaries,RowID,case_location,AnalysisNeighborhoods,CallDateTs,CallYear
210690030,T03,21030278,Alarms,03/10/2021,03/09/2021,03/10/2021 12:16:03 AM,03/10/2021 12:18:36 AM,03/10/2021 12:19:01 AM,03/10/2021 12:20:15 AM,null,null,null,Fire,03/10/2021 12:28:42 AM,1300 Block of MARKET ST,San Francisco,94102,B02,36,3111,3,3,3,false,Alarm,1,TRUCK,3,2,6,Tenderloin,210690030-T03,POINT (-122.41697932641094 37.7770834641944),36,2021-03-10T00:00:00.000+0000,2021
210391607,E19,21017645,Alarms,02/08/2021,02/08/2021,02/08/2021 01:00:14 PM,02/08/2021 01:01:36 PM,02/08/2021 01:01:40 PM,02/08/2021 01:03:21 PM,02/08/2021 01:05:44 PM,null,null,Fire,02/08/2021 01:18:09 PM,400 Block of SERRANO DR,San Francisco,94132,B08,19,8581,3,3,3,true,Alarm,1,ENGINE,1,8,7,Lakeshore,210391607-E19,POINT (-122.48045074945836 37.7190118676788),16,2021-02-08T00:00:00.000+0000,2021
210391164,T04,21017596,Alarms,02/08/2021,02/08/2021,02/08/2021 10:54:56 AM,02/08/2021 10:56:50 AM,02/08/2021 10:56:57 AM,02/08/2021 10:57:07 AM,02/08/2021 10:59:34 AM,null,null,Fire,02/08/2021 11:06:42 AM,600 Block of LONG BRIDGE ST,San Francisco,94158,B03,04,2264,3,3,3,false,Alarm,1,TRUCK,1,3,6,Mission Bay,210391164-T04,POINT (-122.39227179213904 37.77288298280324),4,2021-02-08T00:00:00.000+0000,2021
210683008,T03,21030233,Structure Fire,03/09/2021,03/09/2021,03/09/2021 08:57:47 PM,03/09/2021 08:59:01 PM,03/09/2021 08:59:04 PM,03/09/2021 08:59:28 PM,03/09/2021 09:01:48 PM,null,null,Fire,03/09/2021 09:05:42 PM,600 Block of POLK ST,San Francisco,94102,B02,03,3114,3,3,3,false,Alarm,1,TRUCK,1,2,6,Tenderloin,210683008-T03,POINT (-122.4190191572421 37.78264403568861),36,2021-03-09T00:00:00.000+0000,2021
210391034,E16,21017578,Citizen Assist / Service Call,02/08/2021,02/08/2021,02/08/2021 10:18:53 AM,02/08/2021 10:19:52 AM,02/08/2021 10:19:58 AM,02/08/2021 10:20:42 AM,02/08/2021 10:27:42 AM,null,null,Fire,02/08/2021 10:53:27 AM,FRANKLIN ST/FILBERT ST,San Francisco,94123,B04,16,3233,3,3,3,true,Alarm,1,ENGINE,1,4,2,Marina,210391034-E16,POINT (-122.42581353320875 37.79927566930728),13,2021-02-08T00:00:00.000+0000,2021
210390767,T19,21017552,Other,02/08/2021,02/08/2021,02/08/2021 08:50:27 AM,02/08/2021 08:54:27 AM,02/08/2021 08:55:28 AM,02/08/2021 08:57:51 AM,null,null,null,Fire,02/08/2021 09:02:22 AM,"CALL BOX: JOHN DALY BL/MISSION ST,DC",Daly City,null,B09,33,9922,3,3,3,true,Fire,1,TRUCK,9,None,None,None,210390767-T19,POINT (-122.46239390119047 37.7049649190675),null,2021-02-08T00:00:00.000+0000,2021
210681495,T03,21030060,Alarms,03/09/2021,03/09/2021,03/09/2021 12:47:08 PM,03/09/2021 12:49:37 PM,03/09/2021 12:49:48 PM,03/09/2021 12:51:10 PM,03/09/2021 12:53:01 PM,null,null,Fire,03/09/2021 12:58:21 PM,900 Block of SUTTER ST,San Francisco,94109,B04,03,1557,3,3,3,false,Alarm,1,TRUCK,1,4,3,Nob Hill,210681495-T03,POINT (-122.4160172278388 37.788337139908855),21,2021-03-09T00:00:00.000+0000,2021
210681261,E08,21030034,Alarms,03/09/2021,03/09/2021,03/09/2021 11:37:16 AM,03/09/2021 11:39:15 AM,03/09/2021 11:39:29 AM,03/09/2021 11:40:38 AM,null,null,null,Fire,03/09/2021 11:41:28 AM,100 Block of BRANNAN ST,San Francisco,94107,B03,35,2134,3,3,3,true,Alarm,1,ENGINE,4,3,6,Financial District/South Beach,210681261-E08,POINT (-122.38904057366992 37.78432123494197),8,2021-03-09T00:00:00.000+0000,2021
210382984,B05,21017398,Alarms,02/07/2021,02/07/2021,02/07/2021 09:18:38 PM,02/07/2021 09:20:02 PM,02/07/2021 09:21:15 PM,02/07/2021 09:21:15 PM,02/07/2021 09:25:11 PM,null,null,Fire,02/07/2021 09:35:06 PM,2100 Block of FELL ST,San Francisco,94117,B05,12,4554,3,3,3,false,Alarm,1,CHIEF,2,5,5,Lone Mountain/US

In [0]:
resultPath = "/user/df_fire"
df.write.mode("overwrite").format("parquet").partitionBy("CallYear").parquet(resultPath)